In [425]:
import pandas as pd
import numpy as np

## Charity Regulator Data

In [426]:
ann_reports = pd.read_csv("../Charity_Data/annual-reports-15012021.csv")
pub_reg = pd.read_csv("../Charity_Data/public-register-15012021.csv").drop(['Unnamed: 11'], axis=1)

all_regulator = ann_reports.merge(pub_reg, on='Registered Charity Number', how='left').drop(['Registered Charity Name_y'], axis=1).rename(columns={"Registered Charity Name_x": "Registered Charity Name"})

In [451]:
def include_missing_purpose(data):
    data.loc[(data['Report Activity'].str.contains("Religious activities") | data['Beneficiaries'].str.contains("Religious|Religion|Priests")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of religion"
    
    data.loc[data['Report Activity'].str.contains("Promotion of community") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of community development, including rural or urban regeneration"
    
    data.loc[data['Report Activity'].str.contains("Promotion of health") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Promotion of health, including the prevention or relief of sickness, disease or human suffering"
    
    data.loc[data['Report Activity'].str.contains("Advancement of Arts|Cultural promotion") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of the arts, culture, heritage or sciences"
    
    data.loc[data['Report Activity'].str.contains("Animal welfare") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Prevention or relief of suffering of animals"
    
    data.loc[(data['Report Activity'].str.contains("Welfare/benevolent|Provision of accommodation/housing|Welfare of those in need|Disability support")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of community welfare including the relief of those in need by reason of youth, age, ill-health, or disability"
    
    data.loc[(data['Report Activity'].str.contains("Education|Research/evaluation|Playgroup/afterschool") | data['Beneficiaries'].str.contains("University|School|College")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of education"

    data.loc[(data['Report Activity'].str.contains("Relief of poverty|Overseas aid/famine relief")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Relief of poverty or economic hardship"
    
    if(len(data[data['Charitable Purpose'].isna()]) > 0):
        print("These are the rows without a recoverable match")
        print(data[data['Charitable Purpose'].isna()])
    else:
        print("All rows were successfully processed and now have long-form charitable purposes.")

def condense_purpose(col):
    names = {'Advancement of education': 'Education', ('Promotion of religious or racial harmony and harmonious community relations', 'Advancement of religion'): 'Religion', ('Advancement of community development', 'Other purpose that is of benefit to the community', 'Other purpose that is of benefit to the community;'): 'Community', 'Advancement of the arts, culture, heritage or sciences': 'Arts / Culture', ('Other purpose that is of benefit to the community'): 'Community', 'Prevention or relief of suffering of animals': 'Animals / Environment', 'Promotion of health, including the prevention or relief of sickness, disease or human suffering': 'Health'}

    col.map(names)

In [452]:
include_missing_purpose(all_regulator)
all_regulator['Purpose'] = condense_purpose(all_regulator['Charitable Purpose'].str.split(':').str[0])
all_regulator['Purpose']
#all_regulator.to_csv('../Charity_Data/combined_regulator_data.csv', index=False)

All rows were successfully processed and now have long-form charitable purposes.


0        None
1        None
2        None
3        None
4        None
         ... 
36328    None
36329    None
36330    None
36331    None
36332    None
Name: Purpose, Length: 36333, dtype: object